# Inference
### Your interactive story

-> Try out station

---
### Installation

In [23]:
!python --version

Python 3.9.1


In [24]:
!python -m pip install --upgrade pip

DEPRECATION: tsfresh 0.18.0 has a non-standard dependency specifier matrixprofile>=1.1.10<2.0.0. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of tsfresh or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
!python -m pip install numpy
!python -m pip install torch
!python -m pip install transformers

---
### Imports

In [25]:
import numpy
import torch
import transformers

---
### Initialization

In [26]:
tokenizer = transformers.AutoTokenizer.from_pretrained("gpt2", max_length=883)
tokenizer.add_special_tokens({  "pad_token": "<pad>",
                                "bos_token": "<start>",
                                "eos_token": "<end>"})
tokenizer.add_tokens(["<bot>:"])

1

In [27]:
device = torch.device('cpu')

# loading
#config = transformers.GPT2Config.from_pretrained("gpt2")
#config.max_length = 883
model = transformers.GPT2LMHeadModel.from_pretrained("gpt2", max_length=883, do_sample=True)
model.resize_token_embeddings(len(tokenizer))
model.eval()
model = model.to(device)
model.load_state_dict(torch.load("./model/model_weights_V3_6.pth", map_location=device))

<All keys matched successfully>

In [28]:
start_sentence = "Create me a unique interactive story to calm with the topic:"

In [29]:
out = tokenizer("<start>test<end><pad>")
out

{'input_ids': [50258, 9288, 50259, 50257], 'attention_mask': [1, 1, 1, 1]}

In [30]:
def inference(prompt:str, model, tokenizer, device, padding, clear_output=True):
    prompt = f"<start>{prompt}<bot>:"
    prompt = tokenizer(prompt, return_tensors="pt", padding=padding)
    X = prompt["input_ids"].to(device)
    a = prompt["attention_mask"].to(device)
    output = model.generate(X, attention_mask=a, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(output[0])

    # clean output:
    if clear_output:
        if type(output) == list and len(output) == 1:
            output = output[0]
        elif type(output) == list:
            print("Warning bigger list as Output!")

        if "<bot>:" in output:
            output = "".join(output.split("<bot>:")[1:])

        if "<end>" in output:
            output = "".join(output.split("<end>")[:1])

        if "<pad>" in output:
            output = output.replace("<pad>", "")

    if type(output) == list and len(output) == 1:
        output = output[0]
    return output

---
### Try out

In [31]:
user_input = "ocean"
output = inference(f"{start_sentence} {user_input}", model=model, device=device, tokenizer=tokenizer, padding="max_length", clear_output=True)
output

c:\Users\tobia\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\generation\utils.py:1363: UserWarning: Input length of input_ids is 1024, but `max_length` is set to 883. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


''

In [13]:
loop = 0
while True:
    user_input = input()
    if user_input.lower() in ["q", "quit", "e", "exit", "", "x"]:
        print("See you later! I hope you had fun ^^")
        break
    elif user_input in ["restart", "new"]:
        loop = 0
    if loop == 0:
        user_input = f"{start_sentence} {user_input}"
    print("Bot:", inference(user_input, model=model, device=device, tokenizer=tokenizer, padding="max_length", clear_output=True))
    loop += 1

AttributeError: 'list' object has no attribute 'replace'

---